In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from glob import glob
import json

In [2]:
img_height = 300
img_width = 300

#this is just because of issues with Cyrillic/Unicode characters ion folder names
detail_names_list = ['CS120.01.413', 'CS120.07.442', 'CS150.01.427-01', 'SU160.00.404', 'SU80.01.426', 'SU80.10.409A', 'ЗВТ86.103К-02',
 'СВМ.37.060', 'СВМ.37.060А', 'СВП-120.00.060', 'СВП120.42.020', 'СВП120.42.030', 'СК20.01.01.01.406',
 'СК20.01.01.02.402', 'СК30.01.01.02.402', 'СК30.01.01.03.403', 'СК50.01.01.404', 'СК50.02.01.411', 'СПО250.14.190']

In [3]:
#estimation files should be copid to the folder 'prediction'
path = os.getcwd()
prediction_folder = path + '\\prediction'
folder = prediction_folder + '\\*'
files = glob(folder, recursive = True)
files

['C:\\Users\\Gliwork\\Desktop\\0-Rojects\\2023 Khabarovsk hackaton\\prediction\\IMG_20230920_142557.jpg',
 'C:\\Users\\Gliwork\\Desktop\\0-Rojects\\2023 Khabarovsk hackaton\\prediction\\IMG_20230920_142620.jpg',
 'C:\\Users\\Gliwork\\Desktop\\0-Rojects\\2023 Khabarovsk hackaton\\prediction\\IMG_20230920_142632.jpg',
 'C:\\Users\\Gliwork\\Desktop\\0-Rojects\\2023 Khabarovsk hackaton\\prediction\\IMG_20230920_142640.jpg',
 'C:\\Users\\Gliwork\\Desktop\\0-Rojects\\2023 Khabarovsk hackaton\\prediction\\IMG_20230920_142656.jpg']

In [4]:
#final model name
model_name = 'efficientnet_model.keras'
model_path = path + '\\' + model_name #efficientnet_model.keras
model = tf.keras.models.load_model(model_path)

In [7]:
files = glob(folder, recursive = True)
output_dict = {}
if files!=[]:
    for image_path in files:
        image = tf.keras.utils.load_img(image_path)
        image = tf.image.resize(image, [img_height, img_width])
        input_arr = tf.keras.utils.img_to_array(image)
        predictions = model.predict(np.array([input_arr]), verbose = 0)
        fname = image_path[image_path.rfind('\\')+1:]
        print('файл ', image_path[image_path.rfind('\\')+1:], ' - артикул: ', detail_names_list[np.argmax(predictions)])
        output_dict[fname] = detail_names_list[np.argmax(predictions)]
else:
    print('Проблема: Нет фотографий деталей, проверьте папку "\prediction" ')


файл  IMG_20230920_142557.jpg  - артикул:  СВП-120.00.060
файл  IMG_20230920_142620.jpg  - артикул:  СВП-120.00.060
файл  IMG_20230920_142632.jpg  - артикул:  СВП-120.00.060
файл  IMG_20230920_142640.jpg  - артикул:  СВП-120.00.060
файл  IMG_20230920_142656.jpg  - артикул:  СВП-120.00.060


In [6]:
#write predictions into json file "prediction.json" in folder where ipynb file is located
with open("prediction.json", "w") as file:
    json.dump(output_dict, file)

In [4]:
#skip this cell if just pictures are loaded for prediction - convert video to jpg files (if necessary) - 

import cv2
import time
import os

input_loc = '.....' #name of the videofile

def video_to_frames(input_loc, output_loc):
    """Function to extract frames from input video file
    and save them as separate frames in an output directory.
    Args:
        input_loc: Input video file.
        output_loc: Output directory to save the frames.
    Returns:
        None
    """
    try:
        os.mkdir(output_loc)
    except OSError:
        pass
    # Log the time
    time_start = time.time()
    # Start capturing the feed
    cap = cv2.VideoCapture(input_loc)
    # Find the number of frames
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    print ("Number of frames: ", video_length)
    count = 0
    print ("Converting video..\n")
    # Start converting the video
    while cap.isOpened():
        # Extract the frame
        ret, frame = cap.read()
        if not ret:
            continue
        # Write the results back to output location.
        cv2.imwrite(output_loc + "/%#05d.jpg" % (count+1), frame)
        count = count + 10
        
        # If there are no more frames left
        if (count > (video_length-1)):
            # Log the time again
            time_end = time.time()
            # Release the feed
            cap.release()
            # Print stats
            print ("Done extracting frames.\n%d frames extracted" % count)
            print ("It took %d seconds for conversion." % (time_end-time_start))
            break


            
if __name__=="__main__":

    input_loc = input_loc
    output_loc = prediction_folder
    video_to_frames(input_loc, output_loc)

Number of frames:  -1
Converting video..

